In [1]:
# Cache model example 4 - Testing different replacement strategies
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
# Model parameters
n = 5  # number of items
m = [2, 1]  # cache capacity
alpha = 1.0  # Zipf parameter

# Replacement strategies to test
replStrat = [ReplacementStrategy.RR, ReplacementStrategy.FIFO] #, ReplacementStrategy.LRU]

# Storage for results
AvgTable = []
solver = []
ctmcHitRatio = []
mvaHitRatio = []
ncHitRatio = []

print("repl-strategy, ctmc, mva, nc:")

repl-strategy, ctmc, mva, nc:


In [3]:
# Test each replacement strategy
for s, strategy in enumerate(replStrat):
    # Create model
    model = Network('model')
    source = Source(model, 'Source')
    cacheNode = Cache(model, 'Cache', n, m, strategy)
    sink = Sink(model, 'Sink')
    
    # Create classes
    jobClass = OpenClass(model, 'InitClass', 0)
    hitClass = OpenClass(model, 'HitClass', 0)
    missClass = OpenClass(model, 'MissClass', 0)
    
    # Set arrival
    source.set_arrival(jobClass, Exp(2))
    
    # Zipf-like item references
    pAccess = Zipf(alpha, n)
    cacheNode.set_read(jobClass, pAccess)
    
    # Configure cache classes
    cacheNode.set_hit_class(jobClass, hitClass)
    cacheNode.set_miss_class(jobClass, missClass)
    
    # Routing
    P = model.init_routing_matrix()
    P.set(jobClass, jobClass, source, cacheNode, 1.0)
    P.set(hitClass, hitClass, cacheNode, sink, 1.0)
    P.set(missClass, missClass, cacheNode, sink, 1.0)
    model.link(P)
    
    # Solver CTMC - commented out as runs forever
    ctmc_hr = float('nan')
    # ctmc_solver = SolverCTMC(model, keep=False, cutoff=1, seed=1, verbose=False)
    # solver.append(ctmc_solver)
    # AvgTable.append(ctmc_solver.getAvgNodeTable())
    # hr = cacheNode.hit_ratio()
    # if hr is not None and len(hr) > 0:
    #     ctmc_hr = hr[0] if hasattr(hr, '__len__') else hr
    # else:
    #     ctmc_hr = float('nan')
    ctmcHitRatio.append(ctmc_hr)
    
    # Solver MVA
    try:
        mva_solver = SolverMVA(model, seed=1, verbose=False)
        solver.append(mva_solver)
        model.reset()
        AvgTable.append(mva_solver.getAvgNodeTable())
        hr = cacheNode.hit_ratio()
        if hr is not None and len(hr) > 0:
            mva_hr = hr[0][0] if hasattr(hr[0], '__len__') else hr[0]
        else:
            mva_hr = float('nan')
    except Exception as e:
        mva_hr = float('nan')
    mvaHitRatio.append(mva_hr)

    # Solver NC - try to run it, catch exception if not supported
    try:
        nc_solver = SolverNC(model, seed=1, verbose=False)
        if nc_solver.supports(model):
            solver.append(nc_solver)
            model.reset()
            AvgTable.append(nc_solver.getAvgNodeTable())
            hr = cacheNode.hit_ratio()
            if hr is not None and len(hr) > 0:
                nc_hr = hr[0][0] if hasattr(hr[0], '__len__') else hr[0]
            else:
                nc_hr = float('nan')
        else:
            nc_hr = float('nan')
    except Exception as e:
        nc_hr = float('nan')
    ncHitRatio.append(nc_hr)

     Node   JobClass  QLen  Util  RespT  ResidT    ArvR    Tput
0  Source  InitClass   0.0   0.0    0.0     0.0  0.0000  2.0000
3   Cache  InitClass   0.0   0.0    0.0     0.0  2.0000  0.0000
4   Cache   HitClass   0.0   0.0    0.0     0.0  0.0000  1.3935
5   Cache  MissClass   0.0   0.0    0.0     0.0  0.0000  0.6065
7    Sink   HitClass   0.0   0.0    0.0     0.0  1.3935  0.0000
8    Sink  MissClass   0.0   0.0    0.0     0.0  0.6065  0.0000
     Node   JobClass  QLen  Util  RespT  ResidT    ArvR    Tput
0  Source  InitClass   0.0   0.0    0.0     0.0  0.0000  2.0000
3   Cache  InitClass   0.0   0.0    0.0     0.0  2.0000  0.0000
4   Cache   HitClass   0.0   0.0    0.0     0.0  0.0000  1.4061
5   Cache  MissClass   0.0   0.0    0.0     0.0  0.0000  0.5939
7    Sink   HitClass   0.0   0.0    0.0     0.0  1.4061  0.0000
8    Sink  MissClass   0.0   0.0    0.0     0.0  0.5939  0.0000
     Node   JobClass  QLen  Util  RespT  ResidT    ArvR    Tput
0  Source  InitClass   0.0   0.0    0.0 

In [4]:
# Print results (matching MATLAB format)
print("\\nResults comparison:")
for s, strategy in enumerate(replStrat):
    try:
        strategy_name = ReplacementStrategy.toText(strategy)
    except AttributeError:
        # Fallback if toText method not available
        strategy_name = str(strategy).split('.')[-1].lower()
    
    print(f"{strategy_name}: {ctmcHitRatio[s]:.8f}, {mvaHitRatio[s]:.8f}, {ncHitRatio[s]:.8f}")

\nResults comparison:
rr: nan, 0.69673077, 0.70305657
fifo: nan, 0.69673077, 0.70305657
